In [1]:
#notebook for TD HUU randomization report

In [2]:
# Imports
import os 
import pandas as pd
from django.core.exceptions import ValidationError
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from datetime import date

In [32]:
consents_pids = SubjectConsent.objects.all().values_list('screening_identifier', flat=True).distinct()
consents_pids = list(set(consents_pids))

# TD HUU Caregivers
td_huu_md = MaternalDataset.objects.filter(
    protocol='Tshilo Dikotla',
    mom_hivstatus='HIV-uninfected').values_list('study_maternal_identifier', flat=True)
td_huu_md = list(set(td_huu_md))


# TD Caregivers
td_md = MaternalDataset.objects.filter(
    protocol='Tshilo Dikotla')

#Consented HUU
consented_md = MaternalDataset.objects.filter(
    protocol='Tshilo Dikotla',
    mom_hivstatus='HIV-uninfected',
    screening_identifier__in=consents_pids)
consented_md_pids = consented_md.values_list('study_maternal_identifier', flat=True)
consented_md_pids = list(set(consented_md_pids))

#Unconsented HUU
td_huu_md = MaternalDataset.objects.filter(
    protocol='Tshilo Dikotla',
    mom_hivstatus='HIV-uninfected')
td_huu_md_pids = td_huu_md.values_list('study_maternal_identifier', flat=True)
td_huu_md_pids = list(set(td_huu_md_pids))
td_uncosent_pids = list(set(td_huu_md_pids)-set(consented_md_pids))

# TD HUU Offstudy
off_study_identifiers = CaregiverOffStudy.objects.values_list(
    'subject_identifier', flat=True)
consents = SubjectConsent.objects.filter(
    subject_identifier__in=off_study_identifiers).values_list(
    'screening_identifier', flat=True)
consents = list(set(consents))
offstudy = MaternalDataset.objects.filter(
    protocol='Tshilo Dikotla',
    mom_hivstatus='HIV-uninfected',
    screening_identifier__in=consents).values_list('study_maternal_identifier', flat=True)
offstudy = list(set(offstudy))

data_dict = {
    'TD HUU': td_huu_md,
    'TD HUU Consented': consented_md_pids,
    'TD Not Consented HUU': td_uncosent_pids,
    'TD HUU Offstudy': offstudy
}

print("Total Consented Caregivers: ", len(consents_pids))
print("Total TD Caregivers: ", td_md.count())
print("Total TD HUU Caregivers: ", len(td_huu_md))
print("Total TD Consented HUU: ", len(consented_md_pids))
print("Total TD Not Consented HUU: ", len(td_uncosent_pids))
print("Total TD Offstudy: ", len(offstudy))

Total Consented Caregivers:  1048
Total TD Caregivers:  388
Total TD HUU Caregivers:  114
Total TD Consented HUU:  47
Total TD Not Consented HUU:  67
Total TD Offstudy:  0


In [21]:
# TD Randomised HUU
td_md_identifier = MaternalDataset.objects.filter(
    protocol='Tshilo Dikotla',
    mom_hivstatus='HIV-uninfected').values_list('study_maternal_identifier', flat=True)
td_md_identifier = list(set(td_md_identifier))

randomised_worklist = WorkList.objects.filter(
    assigned__isnull=False,
    study_maternal_identifier__in=td_md_identifier).values_list(
        'study_maternal_identifier', flat=True)
randomised_worklist = list(set(randomised_worklist))

# Not Ranndomised
not_randomised_worklist = WorkList.objects.filter(
    assigned__isnull=True,
    study_maternal_identifier__in=td_md_identifier).values_list(
        'study_maternal_identifier', flat=True)
not_randomised_worklist = list(set(not_randomised_worklist))
final_not_randomised_worklist = []
for ident in not_randomised_worklist:
    log = LogEntry.objects.filter(study_maternal_identifier=ident)
    if not log:
        final_not_randomised_worklist.append(ident)
final_not_randomised_worklist = list(set(final_not_randomised_worklist))

# Missing worklist per prev study
worklist_identifiers = WorkList.objects.filter(
    study_maternal_identifier__in=td_md_identifier).values_list('study_maternal_identifier', flat=True)
worklist_identifiers = list(set(worklist_identifiers))
missing_worklist_identifiers = list(
    set(td_md_identifier) - set(worklist_identifiers))

# Worklist
worklist = WorkList.objects.filter(
    study_maternal_identifier__in=td_md_identifier).values_list(
        'study_maternal_identifier', flat=True)
worklist = list(set(worklist))

data_rando_dict = {
    'Total in the worklist': worklist,
    'Total randomised TD HUU': randomised_worklist,
    'Total Not Randomised TD HUU': final_not_randomised_worklist,
    'Total missing from worklist': missing_worklist_identifiers
}

print("Total in the worklist: ", len(worklist))
print("Total randomised TD HUU: ", len(randomised_worklist))
print("Total Not Randomised TD HUU: ", len(final_not_randomised_worklist))
print("Total missing from worklist: ", len(missing_worklist_identifiers))

Total in the worklist:  114
Total randomised TD HUU:  76
Total Not Randomised TD HUU:  0
Total missing from worklist:  0


In [24]:
# Attempts
td_md_identifier = MaternalDataset.objects.filter(
    protocol='Tshilo Dikotla',
    mom_hivstatus='HIV-uninfected').values_list('study_maternal_identifier', flat=True)
td_md_identifier = list(set(td_md_identifier))

qs = LogEntry.objects.filter(study_maternal_identifier__in=td_md_identifier)
attempt_identifiers = qs.values_list(
        'study_maternal_identifier', flat=True)
attempt_identifiers = list(set(attempt_identifiers))

# Not attempted
not_attempted = list(set(td_md_identifier) - set(attempt_identifiers))

data_attemps_dict = {
    'TD HUU attempts': attempt_identifiers,
    'TD HUU Not attempted': not_attempted
}

print("Attempts made: ", len(attempt_identifiers))
print('No Attempts: ', len(not_attempted))

Attempts made:  113
No Attempts:  1


In [26]:
# Continued Contact
continued_contact = []
for identifier in attempt_identifiers:
    obj = LogEntry.objects.filter(
        study_maternal_identifier=identifier).order_by('-created')
    if obj:
        if obj[0].appt in ['thinking']:
            continued_contact.append(identifier)
        elif obj[0].may_call == 'No' and not obj[0].appt in ['No']:
            continued_contact.append(identifier)
    logs = LogEntry.objects.filter(
        phone_num_success=['none_of_the_above'],
        study_maternal_identifier=identifier).order_by('-created')
    if logs:
        if logs.count() <= 3:
            if logs[0].final_contact == 'No':
                continued_contact.append(identifier)
continued_contact = list(set(continued_contact) - set(consented_md_pids))

data_attemps_dict = {'TD HUU Continued Contact': len(continued_contact)}

print('TD HUU Continued Contact: ', len(continued_contact))

TD HUU Continued Contact:  21


In [27]:
unreacheble = []
for identifier in attempt_identifiers:
    logs = LogEntry.objects.filter(
        phone_num_success=['none_of_the_above'],
        study_maternal_identifier=identifier).order_by('-created')
    if logs.count() >= 3:
        if (logs[0].phone_num_success == ['none_of_the_above'] and
            logs[1].phone_num_success == ['none_of_the_above'] and 
            logs[2].phone_num_success == ['none_of_the_above'] and
            logs[0].final_contact == 'Yes'):
            unreacheble.append(identifier)
unreacheble = list(set(unreacheble) - set(consented_md_pids))


print('TD HUU Unreacheble: ', len(unreacheble))

Unreacheble:  0


In [31]:
# Declined from call log
declined_do_not_call = LogEntry.objects.filter(
    study_maternal_identifier__in=attempt_identifiers,
    may_call__iexact='No').values_list(
        'study_maternal_identifier', flat=True)
declined_do_not_call = list(set(declined_do_not_call))

# Screening rejects
screened_identifiers = ScreeningPriorBhpParticipants.objects.filter(
    flourish_participation='No',
    study_maternal_identifier__in=attempt_identifiers).values_list(
        'study_maternal_identifier', flat=True)
screened_identifiers = list(set(screened_identifiers))

# Merge the 2 above lists
identifiers = screened_identifiers + declined_do_not_call
identifiers = list(set(identifiers) - set(consented_md_pids))

print('TD HUU Declined: ', len(identifiers))

TD HUU Declined:  13
